In [1]:
# Import modules
import pandas as pd
from tabulate import tabulate
# Pre-processing imports
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

In [2]:
# Constants
TRAIN_DATASET = "train.csv"
TEST_DATA = "test.csv"
TEST_LABELS = "test_labels.csv"
DATA_FIELD = ["id","comment_text"]
LABEL_FIELDS = ["id","toxic","severe_toxic","obscene","threat","insult","identity_hate"]
REDUNDANT_FIELDS = ["id"]
STOP_WORDS = set(stopwords.words('english')) # Stopwords in English only

In [3]:
# Read in training dataset
train_dataset = pd.read_csv(TRAIN_DATASET)

# # Split training_data into x_train and y_train -- SAVE FOR LATER
# x_train = training_data[DATA_FIELD]
# y_train = training_data[LABEL_FIELDS]

# Read in test data
test_data = pd.read_csv(TEST_DATA)
test_labels = pd.read_csv(TEST_LABELS)

# Combine test data and labels into one data frame
test_dataset = pd.concat([test_data, test_labels], axis=1)

# Remove redundant id field from both datasets
train_dataset = train_dataset.drop(columns=REDUNDANT_FIELDS)
test_dataset = test_dataset.drop(columns=REDUNDANT_FIELDS)

# Remove samples with labels containing -1 in test dataset, this 
# is a place holder for samples that were not assigned labels.
test_dataset = test_dataset.drop(test_dataset[(test_dataset.toxic == -1) |
                                              (test_dataset.severe_toxic == -1) |
                                              (test_dataset.obscene == -1) |
                                              (test_dataset.threat == -1) |
                                              (test_dataset.insult == -1) |
                                              (test_dataset.identity_hate == -1)].index)

print(train_dataset.head())

                                        comment_text  toxic  severe_toxic  \
0  Explanation\r\nWhy the edits made under my use...      0             0   
1  D'aww! He matches this background colour I'm s...      0             0   
2  Hey man, I'm really not trying to edit war. It...      0             0   
3  "\r\nMore\r\nI can't make any real suggestions...      0             0   
4  You, sir, are my hero. Any chance you remember...      0             0   

   obscene  threat  insult  identity_hate  
0        0       0       0              0  
1        0       0       0              0  
2        0       0       0              0  
3        0       0       0              0  
4        0       0       0              0  


In [4]:
# Remove punctuation (everything except letters and whitespaces)
regex_str = "[^a-zA-Z\s]"
train_dataset['comment_text'] = train_dataset['comment_text'].replace(regex=regex_str, value="")

# Remove extra whitespaces
regex_space = "\s+"
train_dataset['comment_text'] = train_dataset['comment_text'].replace(regex=regex_space, value=" ")

# Strip whitespaces
train_dataset['comment_text'] = train_dataset['comment_text'].str.strip()

# Lowercase
train_dataset['comment_text'] = train_dataset['comment_text'].str.lower()
#print(train_dataset['comment_text'].head())

In [5]:
# Remove stopwords
train_dataset['comment_text'] = train_dataset['comment_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (STOP_WORDS)]))

# Tokenize
train_dataset['comment_text'] = train_dataset['comment_text'].apply(word_tokenize)
#print(train_dataset['comment_text'].head())

In [6]:
porter = PorterStemmer()
train_dataset['comment_text'] = train_dataset['comment_text'].apply(lambda x: [porter.stem(token) for token in x])

In [ ]:
print(train_dataset['comment_text'].head())